<a href="https://colab.research.google.com/github/Garvit996/Data-Science-Garvit/blob/master/Text_Analytics_Movie_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:

!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [7]:
import findspark
findspark.init()

In [8]:
!pip install -q pyspark

     |████████████████████████████████| 212.4 MB 55 kB/s 
     |████████████████████████████████| 198 kB 53.1 MB/s 


In [9]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.functions as F

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
spark

In [11]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-08-19 05:52:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.83.39.146, 34.203.159.69, 52.45.159.115, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.83.39.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  42.4MB/s    in 0.3s    

2021-08-19 05:52:18 (42.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}


In [12]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import *

In [13]:
sc=SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [14]:
from google.colab import files
uploaded = files.upload()

Saving Movies_Dataset_1000.csv to Movies_Dataset_1000.csv


In [15]:
import io
import pandas as pd
# df = pd.read_csv(uploaded['Text_Data.csv'])
# import io
df = pd.read_csv(io.BytesIO(uploaded['Movies_Dataset_1000.csv']))
# df = pd.read_csv(io.StringIO(uploaded['Movies_Dataset.csv'].decode('cp1252')))
# Dataset is now stored in a Pandas Dataframe

In [16]:
df

,id,title,overview
0,0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,1,Jumanji,When siblings Judy and Peter discover an encha...
2,2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...
996,996,The Sword in the Stone,Wart is a young boy who aspires to be a knight...
997,997,So Dear to My Heart,The tale of Jeremiah Kincaid and his quest to ...
998,998,Robin Hood: Prince of Thieves,When the dastardly Sheriff of Nottingham murde...
999,999,Mary Poppins,"The movie combines a diverting story, songs, c..."


In [17]:
# ##---- Testing for df import
# data = spark.read.csv(r'C:/Users/Gravit Agarwal/Downloads/Text_Data.csv',inferSchema=True, header=True)
# ##-----
# documentsPath = "C:/Users/Gravit Agarwal/Downloads/Text_Data.csv"
# documents = sqlContext.read.format("csv").option("header", "true").load(documentsPath)
# ##-----
# #df = spark.read.csv(r"C:/Users/Gravit Agarwal/Downloads/Text_Data.csv", header=True, inferSchema=True)
# df = spark.read.format("csv").option("header", "true").load(r"C:\Users\Gravit Agarwal\Downloads\Text_Data")

In [18]:
df[["id","title","overview"]] = df[["id","title","overview"]].astype(str)

In [19]:
df = spark.createDataFrame(df)

In [20]:
# df = spark.createDataFrame(["The sun is a star","The earth is a planet",
#                             "The moon is a satellite’s earth","The sun is yellow "], "string").toDF("Text")
df.show()        


+---+--------------------+--------------------+
| id|               title|            overview|
+---+--------------------+--------------------+
|  0|           Toy Story|Led by Woody, And...|
|  1|             Jumanji|When siblings Jud...|
|  2|    Grumpier Old Men|A family wedding ...|
|  3|   Waiting to Exhale|Cheated on, mistr...|
|  4|Father of the Bri...|Just when George ...|
|  5|                Heat|Obsessive master ...|
|  6|             Sabrina|An ugly duckling ...|
|  7|        Tom and Huck|A mischievous you...|
|  8|        Sudden Death|International act...|
|  9|           GoldenEye|James Bond must u...|
| 10|The American Pres...|Widowed U.S. pres...|
| 11|Dracula: Dead and...|When a lawyer sho...|
| 12|               Balto|An outcast half-w...|
| 13|               Nixon|An all-star cast ...|
| 14|    Cutthroat Island|Morgan Adams and ...|
| 15|              Casino|The life of the g...|
| 16|Sense and Sensibi...|Rich Mr. Dashwood...|
| 17|          Four Rooms|It's Ted the B

In [21]:
documents = df.withColumn("text_splitted", split(lower(col("overview")), " "))


In [22]:
documents.show()


+---+--------------------+--------------------+--------------------+
| id|               title|            overview|       text_splitted|
+---+--------------------+--------------------+--------------------+
|  0|           Toy Story|Led by Woody, And...|[led, by, woody,,...|
|  1|             Jumanji|When siblings Jud...|[when, siblings, ...|
|  2|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|
|  3|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|
|  4|Father of the Bri...|Just when George ...|[just, when, geor...|
|  5|                Heat|Obsessive master ...|[obsessive, maste...|
|  6|             Sabrina|An ugly duckling ...|[an, ugly, duckli...|
|  7|        Tom and Huck|A mischievous you...|[a, mischievous, ...|
|  8|        Sudden Death|International act...|[international, a...|
|  9|           GoldenEye|James Bond must u...|[james, bond, mus...|
| 10|The American Pres...|Widowed U.S. pres...|[widowed, u.s., p...|
| 11|Dracula: Dead and...|When a l

In [23]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import *
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pyspark.ml.feature import StopWordsRemover 
from pyspark.ml import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
#stopwords.words('english')

In [25]:
documents.count()

1001

In [26]:
documents

DataFrame[id: string, title: string, overview: string, text_splitted: array<string>]

In [27]:
# Remove stop words
remover = StopWordsRemover(inputCol='text_splitted', outputCol='text_splitted_v2')
df_words_no_stopw = remover.transform(documents).select("id","title","overview","text_splitted",'text_splitted_v2')

In [28]:
df_words_no_stopw.show()
#documents.show()

+---+--------------------+--------------------+--------------------+--------------------+
| id|               title|            overview|       text_splitted|    text_splitted_v2|
+---+--------------------+--------------------+--------------------+--------------------+
|  0|           Toy Story|Led by Woody, And...|[led, by, woody,,...|[led, woody,, and...|
|  1|             Jumanji|When siblings Jud...|[when, siblings, ...|[siblings, judy, ...|
|  2|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|[family, wedding,...|
|  3|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|[cheated, on,, mi...|
|  4|Father of the Bri...|Just when George ...|[just, when, geor...|[george, banks, r...|
|  5|                Heat|Obsessive master ...|[obsessive, maste...|[obsessive, maste...|
|  6|             Sabrina|An ugly duckling ...|[an, ugly, duckli...|[ugly, duckling, ...|
|  7|        Tom and Huck|A mischievous you...|[a, mischievous, ...|[mischievous, you...|
|  8|     

In [29]:
# Word2Vec
word2Vec = Word2Vec(vectorSize=100, minCount=1, maxIter=100, inputCol="text_splitted_v2", outputCol="features")
model = word2Vec.fit(df_words_no_stopw)

In [30]:
result = model.transform(df_words_no_stopw)

In [31]:
%pip install mlflow
import mlflow

     |████████████████████████████████| 14.4 MB 64 kB/s 
     |████████████████████████████████| 170 kB 55.4 MB/s 
     |████████████████████████████████| 56 kB 2.9 MB/s 
     |████████████████████████████████| 146 kB 44.8 MB/s 
     |████████████████████████████████| 636 kB 44.0 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 1.1 MB 43.4 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158172 sha256=ed1a0f538f9618a6d0019e1d4340509bd65c58074b068a20a07b70a886978432
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel for databricks-cli: filename=databricks_cli-0.15.0-py3-none-any.whl size=105260 sha256=4cf7d87a736b44aac42d74cea3bbe0d9680a79dffef2f52e6e1967213eb8a28b

In [32]:
mlflow.start_run()

<ActiveRun: >

In [33]:
# result.select("features").show()
# #display(result)

In [34]:
result2 = result.toPandas()

In [35]:
# result2.to_csv('result2.csv',sep = ',',header=True, index=False)

# #mlflow.log_artifact('FinalRerun.csv')

In [36]:
result.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+
| id|               title|            overview|       text_splitted|    text_splitted_v2|            features|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|           Toy Story|Led by Woody, And...|[led, by, woody,,...|[led, woody,, and...|[-0.1197482127922...|
|  1|             Jumanji|When siblings Jud...|[when, siblings, ...|[siblings, judy, ...|[0.27375483000942...|
|  2|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|[family, wedding,...|[-0.4301028016841...|
|  3|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|[cheated, on,, mi...|[-0.4987655802276...|
|  4|Father of the Bri...|Just when George ...|[just, when, geor...|[george, banks, r...|[0.18741645084487...|
|  5|                Heat|Obsessive master ...|[obsessive, maste...|[obsessive, maste...|[-0.3956106736058...|
|

In [37]:
result2

,id,title,overview,text_splitted,text_splitted_v2,features
0,0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[led, by, woody,, andy's, toys, live, happily,...","[led, woody,, andy's, toys, live, happily, roo...","[-0.11974821279220509, 0.4802752844707081, -0...."
1,1,Jumanji,When siblings Judy and Peter discover an encha...,"[when, siblings, judy, and, peter, discover, a...","[siblings, judy, peter, discover, enchanted, b...","[0.2737548300094204, -0.36393830577144404, 0.2..."
2,2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[a, family, wedding, reignites, the, ancient, ...","[family, wedding, reignites, ancient, feud, ne...","[-0.4301028016841773, 0.11192082275043834, -0...."
3,3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[cheated, on,, mistreated, and, stepped, on,, ...","[cheated, on,, mistreated, stepped, on,, women...","[-0.49876558022766276, -0.40855454987493056, -..."
4,4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[just, when, george, banks, has, recovered, fr...","[george, banks, recovered, daughter's, wedding...","[0.18741645084487066, 0.10116111252594877, -0...."
...,...,...,...,...,...,...
996,996,The Sword in the Stone,Wart is a young boy who aspires to be a knight...,"[wart, is, a, young, boy, who, aspires, to, be...","[wart, young, boy, aspires, knight's, squire.,...","[0.06786159289185889, 0.12598968567908742, 0.2..."
997,997,So Dear to My Heart,The tale of Jeremiah Kincaid and his quest to ...,"[the, tale, of, jeremiah, kincaid, and, his, q...","[tale, jeremiah, kincaid, quest, raise, 'champ...","[-0.1057925263359662, -0.3332007431815709, -0...."
998,998,Robin Hood: Prince of Thieves,When the dastardly Sheriff of Nottingham murde...,"[when, the, dastardly, sheriff, of, nottingham...","[dastardly, sheriff, nottingham, murders, robi...","[-0.41394654378808776, -0.020709926326727045, ..."
999,999,Mary Poppins,"The movie combines a diverting story, songs, c...","[the, movie, combines, a, diverting, story,, s...","[movie, combines, diverting, story,, songs,, c...","[0.17948194097398834, 0.04134475924622487, -0...."


In [38]:
#result.withColumn("features", F.regexp_replace(F.regexp_replace(F.regexp_replace("features", "\\]\\[", "-"), "\\[", ""), "\\]", "")).show()
result1 = result.select('features')
#result1 = result.withColumn("features",col("features").cast(StringType()))


In [39]:
column_names = ['col1','col2','col3','col4','col5','col6','col7','col8','col9','col10','col11','col12','col13','col14','col15','col16','col17','col18','col19','col20','col21','col22','col23','col24','col25','col26','col27','col28','col29','col30','col31','col32','col33','col34','col35','col36','col37','col38','col39','col40','col41','col42','col43','col44','col45','col46','col47','col48','col49','col50','col51','col52','col53','col54','col55','col56','col57','col58','col59','col60','col61','col62','col63','col64','col65','col66','col67','col68','col69','col70','col71','col72','col73','col74','col75','col76','col77','col78','col79','col80','col81','col82','col83','col84','col85','col86','col87','col88','col89','col90','col91','col92','col93','col94','col95','col96','col97','col98','col99','col100']

In [40]:
#column_names
temp1 = result1.rdd.map(lambda x:[float(y) for y in x['features']]).toDF(column_names)

In [41]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
#import sparkObject.spark.implicits._
#import org.apache.spark.sql.functions.split

splits = [F.udf(lambda val: float(val[i].item()),FloatType()) for i in range(100)]
temp = result1.select(*[s(F.col('features')).alias(c) for c,s in zip(column_names,splits)])
temp.show()



+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-----------

In [42]:
result1.show()

+--------------------+
|            features|
+--------------------+
|[-0.1197482127922...|
|[0.27375483000942...|
|[-0.4301028016841...|
|[-0.4987655802276...|
|[0.18741645084487...|
|[-0.3956106736058...|
|[-0.6163667181972...|
|[-0.1139660024653...|
|[0.19645283325387...|
|[-0.0987293915823...|
|[0.36204947147483...|
|[-0.1078070867806...|
|[0.27620332464575...|
|[0.04852956852742...|
|[-0.4048826682070...|
|[-0.1125654958188...|
|[0.36740352933605...|
|[0.34709482602775...|
|[-0.7526817582547...|
|[0.24334731435074...|
+--------------------+
only showing top 20 rows



In [ ]:
## Topic Modeling 

In [43]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.linalg.distributed import RowMatrix
#projected.numCols()

In [44]:
# KMeans Clustering
numIterations = 200
numberClusters = 10
kmeans = KMeans().setMaxIter(numIterations).setK(numberClusters).setSeed(1)
kmeans_model = kmeans.fit(result1)

In [45]:
# Make predictions
predictions = kmeans_model.transform(result)

In [46]:
predictions.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|               title|            overview|       text_splitted|    text_splitted_v2|            features|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|           Toy Story|Led by Woody, And...|[led, by, woody,,...|[led, woody,, and...|[-0.1197482127922...|         3|
|  1|             Jumanji|When siblings Jud...|[when, siblings, ...|[siblings, judy, ...|[0.27375483000942...|         3|
|  2|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|[family, wedding,...|[-0.4301028016841...|         0|
|  3|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|[cheated, on,, mi...|[-0.4987655802276...|         0|
|  4|Father of the Bri...|Just when George ...|[just, when, geor...|[george, banks, r...|[0.18741645084487...|         3|
|  5|                Hea

In [47]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

pca = PCA(k=3, inputCol = "features" , outputCol="pcaFeatures")

model = pca.fit(predictions)
transformed_feature = model.transform(predictions)
transformed_feature.select("pcaFeatures").show(truncate=False)

+-----------------------------------------------------------------+
|pcaFeatures                                                      |
+-----------------------------------------------------------------+
|[-0.3673403624632407,-0.32492388773110115,-0.16419603512309744]  |
|[0.1580177370485637,-0.2420259315590702,0.16980499077408387]     |
|[-0.07719928109287677,0.4127977509537516,0.24746039798353414]    |
|[0.23133063487712838,0.11334796447094203,0.24215915816789657]    |
|[-0.12407686987135523,-0.33907482588380383,-0.051776538618981086]|
|[0.21630206347899059,0.22870875565705892,-0.01598417724213524]   |
|[-0.31526620340092215,0.07838106519351624,-0.16743945183494247]  |
|[0.08753678854905456,-0.1451976330688229,-0.6163454820814511]    |
|[0.10154195968597823,0.26260444906144204,0.5528761444392722]     |
|[0.027353117874153864,-0.22085873491485644,0.039955872711085146] |
|[-0.003971286253310848,0.2312315369589749,-0.026233752659198733] |
|[-0.6796353916465383,-0.28725832326664064,-0.09

In [48]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

TF = transformed_feature.select("pcaFeatures")
splits = [F.udf(lambda val: float(val[i].item()),FloatType()) for i in range(1)]
TF1 = TF.select(*[s(F.col('pcaFeatures')).alias(c) for c,s in zip(['col1'],splits)])
TF1.show()
splits = [F.udf(lambda val: float(val[i].item()),FloatType()) for i in range(2)]
TF2 = TF.select(*[s(F.col('pcaFeatures')).alias(c) for c,s in zip(['col1','col2'],splits)])
#TF2.show()
splits = [F.udf(lambda val: float(val[i].item()),FloatType()) for i in range(3)]
TF3 = TF.select(*[s(F.col('pcaFeatures')).alias(c) for c,s in zip(['col1','col2','col3'],splits)])
#TF3.show()

+------------+
|        col1|
+------------+
| -0.36734036|
|  0.15801774|
| -0.07719928|
|  0.23133063|
| -0.12407687|
|  0.21630207|
|  -0.3152662|
|  0.08753679|
|  0.10154196|
| 0.027353117|
|-0.003971286|
|  -0.6796354|
| -0.27483824|
| -0.08687068|
|  0.11539129|
|  0.47820288|
|-0.028043572|
| 0.020054601|
|   0.1817761|
|0.0063133915|
+------------+
only showing top 20 rows



In [49]:
#TF1 = TF1.withColumn("id", monotonically_increasing_id())
#TF2 = TF2.withColumn("id", monotonically_increasing_id())
TF1a = TF1.select(monotonically_increasing_id().alias("id"),"col1")
TF2a = TF2.select(monotonically_increasing_id().alias("id"),"col2")
TF3a = TF3.select(monotonically_increasing_id().alias("id"),"col3")
#TF2a.show()
TF12a = TF1a.join(TF2a, "id", "outer")
TF123a = TF12a.join(TF3a, "id" ,"outer")
Pred = transformed_feature.select("prediction")
Pred = Pred.select(monotonically_increasing_id().alias("id"),"prediction")
TF123a = TF123a.join(Pred, "id" ,"outer")
TF123a.sort("id").show()

+---+------------+-----------+------------+----------+
| id|        col1|       col2|        col3|prediction|
+---+------------+-----------+------------+----------+
|  0| -0.36734036|-0.32492387| -0.16419603|         3|
|  1|  0.15801774|-0.24202593|  0.16980499|         3|
|  2| -0.07719928| 0.41279775|   0.2474604|         0|
|  3|  0.23133063| 0.11334796|  0.24215916|         0|
|  4| -0.12407687|-0.33907482| -0.05177654|         3|
|  5|  0.21630207| 0.22870876|-0.015984178|         5|
|  6|  -0.3152662| 0.07838106| -0.16743945|         3|
|  7|  0.08753679|-0.14519763| -0.61634547|         3|
|  8|  0.10154196| 0.26260445|   0.5528761|         0|
|  9| 0.027353117|-0.22085874| 0.039955873|         5|
| 10|-0.003971286| 0.23123154|-0.026233753|         0|
| 11|  -0.6796354|-0.28725833| -0.09016647|         6|
| 12| -0.27483824| 0.29752192|  -0.6265618|         7|
| 13| -0.08687068| 0.41671047|  -0.4452189|         2|
| 14|  0.11539129|0.048577458|  0.16325179|         3|
| 15|  0.4

In [50]:
TF123a_pandas = TF123a.toPandas()

In [53]:
TF123a_pandas

,id,col1,col2,col3,prediction
0,26,-0.567331,0.475293,-0.261791,2
1,29,-0.159361,-0.250403,-0.556250,2
2,474,-0.352012,-0.103519,0.007673,7
3,964,-0.101836,-0.229032,-0.591126,3
4,65,0.131189,-0.082385,0.039840,5
...,...,...,...,...,...
996,458,-0.224682,-0.516053,-0.454258,5
997,739,0.121402,0.033828,-0.392838,3
998,211,-0.131985,0.080272,-0.261989,7
999,469,0.461927,0.138267,-0.233390,3


In [56]:
import plotly.express as px
df = TF123a_pandas
#fig = px.scatter_3d(TF123a_pandas, x='col1', y='col2', z='col3',color='prediction')
fig = px.scatter_3d(df, x='col1', y='col2', z='col3', size='prediction', color='prediction')
                   # hover_data=  )
fig.update_layout(scene_zaxis_type="log")
fig.show()


In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
df2 = predictions.withColumn("id",col("id").cast(StringType())) \
    .withColumn("title",col("title").cast(StringType())) \
    .withColumn("overview",col("overview").cast(StringType())) \
    .withColumn("text_splitted",col("text_splitted").cast(StringType()))  \
    .withColumn("text_splitted_v2",col("text_splitted_v2").cast(StringType()))  \
    .withColumn("features",col("features").cast(StringType())) \
    .withColumn("prediction",col("prediction").cast(StringType()))
df2.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- text_splitted: string (nullable = true)
 |-- text_splitted_v2: string (nullable = true)
 |-- features: string (nullable = true)
 |-- prediction: string (nullable = false)



In [ ]:
df4 = df2.select('id','title','features',"prediction")

In [ ]:
from pyspark import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark import SparkConf, SparkContext

commaToDot = udf(lambda x : float(str(x).replace(',', '.')), FloatType())
# df2=df.withColumn('area',commaToDot(df._c0))

df3 = predictions.withColumn('id', regexp_replace('id', ',', '.').cast('String')) \
    .withColumn('title', regexp_replace('title', ',', '.').cast('String')) \
    .withColumn('overview', regexp_replace('overview', ',', ' ').cast('String')) \
    .withColumn('text_splitted', regexp_replace('text_splitted', ',', ' ').cast('String'))  \
    .withColumn('text_splitted_v2', regexp_replace('text_splitted_v2', ',', ' ').cast('String')) \
    .withColumn('features').withColumn('prediction', regexp_replace('prediction', ',', '.').cast('String'))
df3.printSchema()

TypeError: ignored

In [ ]:
df3.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|               title|            overview|       text_splitted|    text_splitted_v2|            features|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|           Toy Story|Led by Woody  And...|[led by woody  an...|org.apache.spark....|[6,1,0,0,28000003...|         2|
|  1|             Jumanji|When siblings Jud...|[when siblings ju...|org.apache.spark....|[6,1,0,0,28000003...|         2|
|  2|    Grumpier Old Men|A family wedding ...|[a family wedding...|org.apache.spark....|[6,1,0,0,28000003...|         4|
|  3|   Waiting to Exhale|Cheated on  mistr...|[cheated on  mist...|org.apache.spark....|[6,1,0,0,28000003...|         2|
|  4|Father of the Bri...|Just when George ...|[just when george...|org.apache.spark....|[6,1,0,0,28000003...|         0|
|  5|                Hea

In [ ]:
result

DataFrame[id: string, title: string, overview: string, text_splitted: array<string>, text_splitted_v2: array<string>, features: vector]

In [ ]:
#predictions1[["id","title","overview","text_splitted","text_splitted_v2","features","prediction"]] = predictions[["id","title","overview","text_splitted","text_splitted_v2","features","prediction"]].astype(str)

In [ ]:
result.write.csv("predictions_Movie_5thAug_v3.csv", mode="overwrite", header = "true")
# ,
#   delimiter = ",",
#   quote = "\"",
#   escape = "\\",
#   charset = "UTF-8")

AnalysisException: ignored

In [ ]:
df2.show()

In [ ]:
# Save Output
predictionsPath = "../data/predictions_Movie_v2.csv"
df2.select(["id","title","overview","text_splitted","text_splitted_v2","features", "prediction"]).write.csv(predictionsPath, mode="overwrite", header="true")

In [ ]:

# output the result to a text file.

clusters = df.groupby('cluster')    

for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w') # create csv file
    data = clusters.get_group(cluster)[['title','overview']] # get title and overview columns
    f.write(data.to_csv(index_label='id')) # set index to id
    f.close()

print("Cluster centroids: \n")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(k):
    print("Cluster %d:" % i)
    for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
        print (' %s' % terms[j])
    print('------------')


In [ ]:
# KMeans Clustering
numIterations = 200
numberClusters = 2
kmeans = KMeans().setMaxIter(numIterations).setK(numberClusters).setSeed(1)
kmeans_model = kmeans.fit(result)

In [ ]:
# Make predictions
predictions = kmeans_model.transform(result)

In [ ]:
predictions.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|               title|            overview|       text_splitted|    text_splitted_v2|            features|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0|           Toy Story|Led by Woody, And...|[led, by, woody,,...|[led, woody,, and...|[0.32688762738623...|         6|
|  1|             Jumanji|When siblings Jud...|[when, siblings, ...|[siblings, judy, ...|[0.13762119575403...|         0|
|  2|    Grumpier Old Men|A family wedding ...|[a, family, weddi...|[family, wedding,...|[0.25666033335481...|         2|
|  3|   Waiting to Exhale|Cheated on, mistr...|[cheated, on,, mi...|[cheated, on,, mi...|[0.14437834605381...|         8|
|  4|Father of the Bri...|Just when George ...|[just, when, geor...|[george, banks, r...|[-0.2310195296174...|         0|
|  5|                Hea